In [41]:
import xml.etree.ElementTree as ET

In [84]:
root = ET.parse('../db/all_courses.xml')

In [1]:
for i, item in enumerate(root.findall('course')):
    courses[i] = dict()
    if item.tag == "course": 
        for node in item:
            courses[i]['course_id'] = node.find('id').text
            courses[i]['course_level'] = node.find('level').text       
            courses[i]['course_name'] = node.find('name').text            
            courses[i]['description']= node.find('description').text
            courses[i]['parent'] = node.find('parent').text
            
            if not int(courses[i]['course_level'][1:]) == 100:  #non-leaf node
                #print(node.find('name').text)
                courses[i]['node_placement'] = 'non-leaf'
                courses[i]['children'] = dict()
                children = node.find('children')
                for j, child in enumerate(children.findall('child')):
                    child_type = child.find('child_type').text
                    #print(child_type)
                    child_name = child.find('child_name').text
                    courses[i]['children'][j] = {'child_type':child_type, 'child_name':child_name}
                #print(courses[i]['children'])
            
            if int(courses[i]['course_level'][1:]) == 100:  #leaf node
                
                courses[i]['node_placement'] = 'leaf'
                courses[i]['duration'] =  node.find('duration').text
                courses[i]['sector'] =  node.find('sector').text
                colleges = node.find('offline')
                courses[i]['offline'] = []
                for col in colleges.findall('college'):
                    courses[i]['offline'].append(col.text)
                
                
                
                
            
            
            
            
            

NameError: name 'root' is not defined

In [90]:
courses

{0: {'course_id': 'C10',
  'course_level': 'X0',
  'course_name': '10th',
  'description': '10th Grade',
  'parent': None,
  'node_placement': 'non-leaf',
  'children': {0: {'child_type': 'course',
    'child_name': 'Industrial Training'},
   1: {'child_type': 'course', 'child_name': 'Intermediate (11th/12th)'},
   2: {'child_type': 'course', 'child_name': 'Polytechnic'}}},
 1: {'course_id': 'CITI',
  'course_level': 'A1',
  'course_name': 'Industrial Training',
  'description': 'Industrial Training (ITIs)',
  'parent': '10th',
  'node_placement': 'non-leaf',
  'children': {0: {'child_type': 'course',
    'child_name': 'Computer Operator (COPA)'},
   1: {'child_type': 'course', 'child_name': 'Fitter'},
   2: {'child_type': 'course', 'child_name': 'Welder'},
   3: {'child_type': 'course', 'child_name': 'Electrician'},
   4: {'child_type': 'course', 'child_name': 'Motor Mechanic'},
   5: {'child_type': 'course', 'child_name': 'Tourist Guide'}}},
 2: {'course_id': 'CITIComputer',
  'cours

In [95]:
for c in courses.keys():
    if courses[c]['course_name'] == 'Industrial Training':
        print(courses[c])

{'course_id': 'CITI', 'course_level': 'A1', 'course_name': 'Industrial Training', 'description': 'Industrial Training (ITIs)', 'parent': '10th', 'node_placement': 'non-leaf', 'children': {0: {'child_type': 'course', 'child_name': 'Computer Operator (COPA)'}, 1: {'child_type': 'course', 'child_name': 'Fitter'}, 2: {'child_type': 'course', 'child_name': 'Welder'}, 3: {'child_type': 'course', 'child_name': 'Electrician'}, 4: {'child_type': 'course', 'child_name': 'Motor Mechanic'}, 5: {'child_type': 'course', 'child_name': 'Tourist Guide'}}}


In [5]:
import pandas as pd
NCS_CAREER_PATHS = pd.read_excel('./CAREERS_500.xlsx')
Y_FILLED_SECTORS = NCS_CAREER_PATHS['Sector'].fillna(method='ffill', axis=0)
Y_FILLED_SECTOR_DESC = NCS_CAREER_PATHS['Sector Description'].fillna(method='ffill', axis=0)
ncs_sector_names = list(Y_FILLED_SECTORS.unique())
NCS_CAREER_PATHS['Sector'] = Y_FILLED_SECTORS
NCS_CAREER_PATHS['Sector Description'] = Y_FILLED_SECTOR_DESC
ncs_career_names = list(NCS_CAREER_PATHS['Career Name'].unique())
ncscourses = dict()
for i, ncs_sector_name in enumerate(ncs_sector_names):
    ncscourses[i] = dict()
    ncscourses[i]['course_id'] = 'C'+"".join(ncs_sector_name.split(" ")).strip()   
    ncscourses[i]['course_level'] = "D3"
    ncscourses[i]['course_name'] = ncs_sector_name
    ncscourses[i]['description']= str(NCS_CAREER_PATHS[NCS_CAREER_PATHS['Sector'] == ncs_sector_name]['Sector Description'].tolist()[0])
    ncscourses[i]['parent'] = "Intermediate (11th/12th)"
    ncscourses[i]['node_placement'] = 'non-leaf'
    if str(ncs_sector_name) == "nan":
        continue
    
    courses_in_ncs_sector = NCS_CAREER_PATHS[NCS_CAREER_PATHS['Sector'] == ncs_sector_name]['Career Name'].tolist().copy()
    ncscourses[i]['children'] = dict()
    for jx, cx in enumerate(courses_in_ncs_sector):
        ncscourses[i]['children'][jx] = {'child_type':'ncscourse', 'child_name':", ".join(cx.split("/"))}

          
            
for k in range(i+1, NCS_CAREER_PATHS.shape[0]+(i+1)):
    ncscourses[k] = dict()
    ncscourses[k]['course_id'] = 'C_NCS_ENDPOINT'+str(k)  
    ncscourses[k]['course_level'] = "D100"
    ncscourses[k]['course_name'] = ", ".join(NCS_CAREER_PATHS['Career Name'].iloc[k-i-1].split("/"))
    ncscourses[k]['description']= NCS_CAREER_PATHS['Career Description'].iloc[k-i-1]
    ncscourses[k]['parent'] = NCS_CAREER_PATHS['Sector'].iloc[k-i-1]
    ncscourses[k]['node_placement'] = 'leaf'
    ncscourses[k]['sector'] =  NCS_CAREER_PATHS['Sector'].iloc[k-i-1]
    ncscourses[k]['offline'] =  NCS_CAREER_PATHS['Where will you study?'].iloc[k-i-1]
    ncscourses[k]['duration'] = ''
    ncscourses[k]['details'] = {'personalCompetencies': NCS_CAREER_PATHS['Personal Competencies'].iloc[k-i-1], 
                                'whereToWork': NCS_CAREER_PATHS['Where will you work?'].iloc[k-i-1],
                                'expectedGrowthPath': NCS_CAREER_PATHS['Expected Growth Path'].iloc[k-i-1],
                                'fees': NCS_CAREER_PATHS['Fees'].iloc[k-i-1],
                                'scholarshipsAndLoans': NCS_CAREER_PATHS['Scholarships & Loans'].iloc[k-i-1],
                                'expectedIncome': NCS_CAREER_PATHS['Expected Income'].iloc[k-i-1],
                                'externalLink': NCS_CAREER_PATHS['NCS Link'].iloc[k-i-1]
                                
                               }


FileNotFoundError: [Errno 2] No such file or directory: './CAREERS_500.xlsx'

In [6]:
!ls

.ipynb_checkpoints
Courses
Exams
StudentOptions
all_courses.xml
Untitled.ipynb
